Измалкова Дарья БКЛ182
# NLP Домашнее задание № 2

In [1]:
import pandas as pd
import gzip
import json
import re

import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.collocations import *
from nltk.metrics.spearman import *


import spacy
from spacy import displacy
from spacy.matcher import Matcher
import en_core_web_sm

from collections import Counter

In [2]:
stop_words = set(stopwords.words('english'))
nlp = en_core_web_sm.load()

## Извлекаю данные
Возьму корпус бытовой техники, подкорпус товаров Bosch

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [4]:
metadata = getDF('meta_Appliances.json.gz')

In [5]:
reviews = getDF('Appliances.json.gz')

In [6]:
meta = metadata[['title', 'asin', 'brand']]
reviews = reviews[['reviewText', 'asin']]

Список товаров не совпадал, попались, кажется, еще какие-то книги - убираю

In [7]:
in_meta = set(meta['asin'].unique())
in_rev = set(reviews['asin'].unique())

mis = in_rev - in_meta

In [8]:
reviews = reviews[~reviews['asin'].isin(mis)]

В итоге оставляю стобцы с отзывом, названием товара и брендом 

In [9]:
merg = (reviews.merge(meta, left_on='asin', right_on='asin')
          .reindex(columns=['title', 'reviewText', 'brand']))
merg['reviewText'] = merg['reviewText'].apply(str)

In [10]:
bosch = merg[merg['brand'] == 'Bosch']

In [11]:
bosch

,title,reviewText,brand
71626,Bosch 093046USE 093406,Perfect replacement! Same material and build a...,Bosch
71627,Bosch 093046USE 093406,Our new home came with a Bosch already install...,Bosch
71628,Bosch 093046USE 093406,"It's a little flimsy, so hopefully it holds up...",Bosch
71629,Bosch 093046USE 093406,It is an exact replacement for the tray that c...,Bosch
71630,Bosch 093046USE 093406,Fit perfectly!,Bosch
71631,Bosch 093046USE 093406,This dishwasher utensil basket was a perfect r...,Bosch
71632,Bosch 093046USE 093406,This isn't exactly the correct basket for my 2...,Bosch
71633,Bosch 093046USE 093406,Perfect replacement,Bosch
71634,Bosch 093046USE 093406,A+,Bosch
71635,Bosch 093046USE 093406,Great cutlery basket to replace what I had in ...,Bosch


## Способы выделить названия товаров
* Можно собрать подкорпус названий товаров или ожидаемых слов (+ расширить его с помощью векторов близких по смыслу слов w2v или WordNet) "
* или составить правила с помошью yargy
* или Bootstrping: составить шаблоны вида "XX dishwasher"
* или воспользоваться готовыми инструментами извлечения именнованных сущностей, например spacy

Правила и bootstraping требуют хотя бы какого-то знакомства с данными и больше ручной работы. Готовые инструменты не подстроены под конкретные задачи, в нашем случае будут выделять кучу ненужных слов, например географические места, и не выделять простые ИГ, dishwasher, refrigerator etc. Проще всего воспользоваться готовыми названиями, но в них содержиться много лишней информации, плюс, в отзывах часто не так конкретно называют товар ("this dishwasher basket" а не "Bosch 093046USE 093406")


#### Я решила сделать комбинацию из обработки названий товаров и NE, которые выделяет spacy. 

воспользуюсь лемматизированным текстом

In [12]:
def lemmatize(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]','',text)
    text = text.replace('.','')
    tokens = nlp(text)
    lemmas = [word.lemma_ for word in tokens
                      if word.lemma_ not in stop_words]
    return lemmas

In [13]:
bosch['spacy'] = bosch['reviewText'].apply(nlp)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
bosch['lem'] = bosch['reviewText'].apply(lemmatize)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
def extract_ne(list_of_words):
    nes = set()
    text = ' '.join(list_of_words)
    doc = nlp(text)
    for ent in doc.ents:   
        if ent.label_ in ['DATE','TIME', 'CARDINAL', 'GPE', 'LOC', 'MONEY', 'ORDINAL', 'QUANTITY', 'PERCENT', 'NORP']:
            pass
        else:
            nes.update(ent)
    return list(nes)

In [16]:
bosch['lem_nes'] = bosch['lem'].apply(extract_ne)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


В названиях товаров много цифр и информации по типу материала, функциях итп, поэтому выделю из них вершины именных групп.

In [17]:
def extract_nouns(list_of_words):
    text = ' '.join(list_of_words)
    doc = nlp(text)
    nps = []
    nps.extend([chunk.root.lemma_ for chunk in doc.noun_chunks])
    return nps

In [18]:
bosch['title_lem'] = bosch['title'].apply(lemmatize)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
bosch['title_nps'] = bosch['title_lem'].apply(extract_nouns)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
bosch['lem_nes'] = bosch.apply(lambda x: x.lem_nes + x.title_nps, axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
bosch

,title,reviewText,brand,spacy,lem,lem_nes,title_lem,title_nps
71626,Bosch 093046USE 093406,Perfect replacement! Same material and build a...,Bosch,"(Perfect, replacement, !, Same, material, and,...","[perfect, replacement, material, build, one, c...",[bosch],"[bosch, 093046use, 093406]",[bosch]
71627,Bosch 093046USE 093406,Our new home came with a Bosch already install...,Bosch,"(Our, new, home, came, with, a, Bosch, already...","[-PRON-, new, home, come, bosch, already, inst...",[bosch],"[bosch, 093046use, 093406]",[bosch]
71628,Bosch 093046USE 093406,"It's a little flimsy, so hopefully it holds up...",Bosch,"(It, 's, a, little, flimsy, ,, so, hopefully, ...","[-PRON-, little, flimsy, hopefully, -PRON-, ho...",[bosch],"[bosch, 093046use, 093406]",[bosch]
71629,Bosch 093046USE 093406,It is an exact replacement for the tray that c...,Bosch,"(It, is, an, exact, replacement, for, the, tra...","[-PRON-, exact, replacement, tray, come, -PRON...",[bosch],"[bosch, 093046use, 093406]",[bosch]
71630,Bosch 093046USE 093406,Fit perfectly!,Bosch,"(Fit, perfectly, !)","[fit, perfectly]",[bosch],"[bosch, 093046use, 093406]",[bosch]
71631,Bosch 093046USE 093406,This dishwasher utensil basket was a perfect r...,Bosch,"(This, dishwasher, utensil, basket, was, a, pe...","[dishwasher, utensil, basket, perfect, replace...",[bosch],"[bosch, 093046use, 093406]",[bosch]
71632,Bosch 093046USE 093406,This isn't exactly the correct basket for my 2...,Bosch,"(This, is, n't, exactly, the, correct, basket,...","[exactly, correct, basket, -PRON-, 2008, bosch...",[bosch],"[bosch, 093046use, 093406]",[bosch]
71633,Bosch 093046USE 093406,Perfect replacement,Bosch,"(Perfect, replacement)","[perfect, replacement]",[bosch],"[bosch, 093046use, 093406]",[bosch]
71634,Bosch 093046USE 093406,A+,Bosch,(A+),[],[bosch],"[bosch, 093046use, 093406]",[bosch]
71635,Bosch 093046USE 093406,Great cutlery basket to replace what I had in ...,Bosch,"(Great, cutlery, basket, to, replace, what, I,...","[great, cutlery, basket, replace, -PRON-, miel...","[washer, bosch]","[bosch, 093046use, 093406]",[bosch]


В качестве именновонных сущностей возьму 50 самых частотых слов, которые выделились из названий и которые выделил spacy

In [22]:
extracted = []
for nes in bosch['lem_nes']:
        extracted.extend(nes)

In [23]:
counted = Counter(extracted)
extracted = [word[0] for word in counted.most_common(50)]

In [24]:
# не идеально, но что получилось...
extracted

['bosch',
 'washer',
 'dishwasher',
 'star',
 'series',
 'cooktop',
 'steel',
 'wheel',
 'dryer',
 'gasket',
 'part',
 'load',
 'door',
 'convection',
 'cu',
 'pump',
 'drawer',
 'white',
 'range',
 'ascenta',
 'place',
 'cord',
 'filter',
 'slidein',
 'nexxt',
 'pumpdrain',
 'stainless',
 'paykel',
 'dd605ss',
 'micro',
 'wte86300usaxxis',
 'plat',
 'latch',
 'capacity',
 'she3ar55uc',
 'ngm8655uc',
 'panelfacia',
 'wall',
 'sleeve',
 'aquastop',
 'rate',
 'module',
 'unit',
 'tub',
 'kit',
 'wfmc2201uc',
 'black',
 'wtv76100usaxxis',
 'oven',
 'mesh']

## Выделение биграмм и триграмм и ранжирование с помощью коллакационных метрик

In [25]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [26]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [27]:
left_context = [{'IS_PUNCT': False,'IS_STOP' : False}, {'LEMMA': {'IN': extracted}}]
right_context = [{'LEMMA': {'IN': extracted}}, {'IS_PUNCT': False, 'IS_STOP' : False}]

context_matcher = Matcher(nlp.vocab)
context_matcher.add('left_context', None, left_context)
context_matcher.add('right_context', None, right_context)

In [29]:
left_context = [{'IS_PUNCT': False, 'IS_STOP' : False}, 
                {'IS_PUNCT': False, 'IS_STOP' : False}, 
                {'LEMMA': {'IN': extracted}}]
right_context = [{'LEMMA': {'IN': extracted}}, 
                 {'IS_PUNCT': False, 'IS_STOP' : False},
                 {'IS_PUNCT': False, 'IS_STOP' : False}]

context_matcher2 = Matcher(nlp.vocab)
context_matcher2.add('left_context', None, left_context)
context_matcher2.add('right_context', None, right_context)

In [30]:
def create_ngram_list(matcher,docs):
    collocs = []
    for doc in docs:
        matches = matcher(doc)
        for match_id, start, end in matches:
            span = doc[start:end]
            collocs.append(span.text.lower().split())
    return collocs

In [31]:
collocs2 = create_ngram_list(context_matcher, bosch['spacy'])

In [32]:
collocs3 = create_ngram_list(context_matcher2, bosch['spacy'])

In [33]:
finder2 = BigramCollocationFinder.from_documents(collocs2)
finder3 = TrigramCollocationFinder.from_documents(collocs3)

In [34]:
bi_scores_ll = finder2.score_ngrams(bigram_measures.likelihood_ratio)
bi_scores_pmi = finder2.score_ngrams(bigram_measures.pmi)
bi_scores_mi_like = finder2.score_ngrams(bigram_measures.mi_like)
bi_scores_student_t = finder2.score_ngrams(bigram_measures.student_t)

In [35]:
tri_scores_ll = finder3.score_ngrams(trigram_measures.likelihood_ratio)
tri_scores_pmi = finder3.score_ngrams(trigram_measures.pmi)
tri_scores_mi_like = finder3.score_ngrams(trigram_measures.mi_like)
tri_scores_student_t = finder3.score_ngrams(trigram_measures.student_t)

### Ранки биграмм

In [36]:
list(ranks_from_scores(bi_scores_pmi))[:20]

[(('angles', 'ranging'), 0),
 (('appropriate', 'cords'), 0),
 (('tentatively', 'steeling'), 0),
 (('aftermarket', 'pumps'), 3),
 (('placing', 'spatulas'), 3),
 (('watch', 'placing'), 3),
 (('temp', 'sleeves'), 6),
 (('long', 'sleeve'), 7),
 (('sleeve', 'dress'), 7),
 (('slowly', 'pumping'), 7),
 (('bother', 'filtering'), 10),
 (('filtering', 'ability'), 10),
 (('places', 'wanted'), 10),
 (('weird', 'places'), 10),
 (('brass', 'sleeve'), 14),
 (('consistently', 'latched'), 15),
 (('longer', 'latched'), 15),
 (('pumps', 'etc'), 15),
 (('pumps', 'provides'), 15),
 (('twin', 'tubs'), 15)]

In [37]:
list(ranks_from_scores(bi_scores_ll))[:20]

[(('stainless', 'steel'), 0),
 (('bosch', 'dishwasher'), 1),
 (('door', 'open'), 2),
 (('replacement', 'part'), 3),
 (('800', 'series'), 4),
 (('5', 'stars'), 5),
 (('drain', 'pump'), 6),
 (('gas', 'cooktop'), 7),
 (('power', 'cord'), 8),
 (('warming', 'drawer'), 9),
 (('dryer', 'set'), 10),
 (('steel', 'tub'), 11),
 (('small', 'loads'), 12),
 (('water', 'filter'), 13),
 (('place', 'settings'), 14),
 (('500', 'series'), 15),
 (('door', 'seal'), 16),
 (('part', 'number'), 17),
 (('rubber', 'gasket'), 18),
 (('dish', 'washer'), 19)]

In [38]:
list(ranks_from_scores(bi_scores_mi_like))[:20]

[(('stainless', 'steel'), 0),
 (('bosch', 'dishwasher'), 1),
 (('5', 'stars'), 2),
 (('power', 'cord'), 3),
 (('800', 'series'), 4),
 (('door', 'open'), 5),
 (('replacement', 'part'), 6),
 (('drain', 'pump'), 7),
 (('warming', 'drawer'), 8),
 (('place', 'settings'), 9),
 (('small', 'loads'), 10),
 (('gas', 'cooktop'), 11),
 (('induction', 'cooktops'), 12),
 (('500', 'series'), 13),
 (('highly', 'rated'), 14),
 (('rubber', 'gasket'), 15),
 (('old', 'dishwasher'), 16),
 (('steel', 'tub'), 17),
 (('dryer', 'set'), 18),
 (('water', 'filter'), 19)]

In [39]:
list(ranks_from_scores(bi_scores_student_t))[:20]

[(('stainless', 'steel'), 0),
 (('bosch', 'dishwasher'), 1),
 (('door', 'open'), 2),
 (('replacement', 'part'), 3),
 (('old', 'dishwasher'), 4),
 (('bosch', 'washer'), 5),
 (('new', 'dishwasher'), 6),
 (('steel', 'tub'), 7),
 (('water', 'filter'), 8),
 (('gas', 'cooktop'), 9),
 (('door', 'latch'), 10),
 (('bosch', 'dishwashers'), 11),
 (('800', 'series'), 12),
 (('5', 'stars'), 13),
 (('door', 'seal'), 14),
 (('drain', 'pump'), 15),
 (('door', 'gasket'), 16),
 (('dish', 'washer'), 17),
 (('dryer', 'set'), 18),
 (('small', 'loads'), 19)]

### Ранки триграмм

In [40]:
list(ranks_from_scores(tri_scores_pmi))[:20]

[(('bottomed', 'cups', 'placed'), 0),
 (('crusty', 'utensils', 'placed'), 0),
 (('included', 'metal', 'gaskets'), 0),
 (('sleeve', 'dress', 'shirts'), 0),
 (('higher', 'flow', 'rate'), 4),
 (('b22cs50sns', '22', 'cu'), 5),
 (('b26ft70sns', '25.9', 'cu'), 5),
 (('loaded', 'retainer', 'rings'), 7),
 (('pumps', 'provides', 'better'), 7),
 (('20', 'hookup', 'kit'), 9),
 (('filters', 'positively', 'suck'), 10),
 (('latest', 'reviews', 'rates'), 10),
 (('little', 'brass', 'sleeve'), 10),
 (('ordering', 'generic', 'filters'), 10),
 (('shallow', 'cutlery', 'drawer'), 10),
 (('day', 'cooking', 'ranges'), 15),
 (('heavy', 'duty', 'cord'), 15),
 (('shows', 'glasses', 'loaded'), 17),
 (('latch', 'pieces', 'meet'), 18),
 (('local', 'appliance', 'places'), 18)]

In [41]:
list(ranks_from_scores(tri_scores_ll))[:20]

[(('stainless', 'steel', 'tub'), 0),
 (('stainless', 'steel', 'interior'), 1),
 (('stainless', 'steel', 'design'), 2),
 (('stainless', 'steel', 'outer'), 3),
 (('stainless', 'steel', 'appliances'), 4),
 (('stainless', 'steel', 'mesh'), 5),
 (('3-', 'stainless', 'steel'), 6),
 (('resistant', 'stainless', 'steel'), 6),
 (('stainless', 'steel', 'panels'), 8),
 (('stainless', 'steel', 'nemesis'), 9),
 (('stainless', 'steel', 'pots'), 9),
 (('stainless', 'steel', 'stains'), 9),
 (('stainless', 'steel', 'tubs'), 9),
 (('stainless', 'steel', 'exterior'), 13),
 (('stainless', 'steel', 'panel'), 14),
 (('piece', 'stainless', 'steel'), 15),
 (('stainless', 'steel', 'door'), 16),
 (('beautiful', 'stainless', 'steel'), 17),
 (('stainless', 'steel', 'finish'), 17),
 (('stainless', 'steel', 'cleaner'), 19)]

In [42]:
list(ranks_from_scores(tri_scores_mi_like))[:20]

[(('bottomed', 'cups', 'placed'), 0),
 (('crusty', 'utensils', 'placed'), 0),
 (('included', 'metal', 'gaskets'), 0),
 (('sleeve', 'dress', 'shirts'), 0),
 (('higher', 'flow', 'rate'), 4),
 (('b22cs50sns', '22', 'cu'), 5),
 (('b26ft70sns', '25.9', 'cu'), 5),
 (('lp', 'conversion', 'kit'), 7),
 (('loaded', 'retainer', 'rings'), 8),
 (('pumps', 'provides', 'better'), 8),
 (('20', 'hookup', 'kit'), 10),
 (('filters', 'positively', 'suck'), 11),
 (('latest', 'reviews', 'rates'), 11),
 (('little', 'brass', 'sleeve'), 11),
 (('ordering', 'generic', 'filters'), 11),
 (('shallow', 'cutlery', 'drawer'), 11),
 (('drawers', 'makes', 'loading'), 16),
 (('day', 'cooking', 'ranges'), 17),
 (('heavily', 'soiled', 'loads'), 17),
 (('heavy', 'duty', 'cord'), 17)]

In [43]:
list(ranks_from_scores(tri_scores_student_t))[:20]

[(('stainless', 'steel', 'tub'), 0),
 (('new', 'bosch', 'dishwasher'), 1),
 (('year', 'old', 'dishwasher'), 2),
 (('bosch', '800', 'series'), 3),
 (('old', 'bosch', 'dishwasher'), 4),
 (('bosch', 'induction', 'cooktop'), 5),
 (('exact', 'replacement', 'part'), 6),
 (('bosch', 'customer', 'service'), 7),
 (('500', 'series', 'washer'), 8),
 (('bosch', '500', 'series'), 9),
 (('bosch', 'dish', 'washer'), 10),
 (('drawers', 'makes', 'loading'), 11),
 (('paykel', 'dish', 'drawers'), 12),
 (('unit', 'immediately', 'discolored'), 12),
 (('place', 'setting', 'capacity'), 14),
 (('washers', 'high', 'capacity'), 15),
 (('part', 'fit', 'perfectly'), 16),
 (('bosch', 'clothes', 'washer'), 17),
 (('stainless', 'steel', 'interior'), 18),
 (('bosch', 'french', 'door'), 19)]

С биграммами, кажется, слегка хуже pmi, остальные примерно одинакого ранжируют. С триграммами хуже всех loglikelihood и лучше, кажется, student t-test.

## Топ-5 н-грамм для какого-то товара

In [44]:
def get_top_n_bigram(scores, word, n):
    collocs = []
    for bigram in scores:
        if bigram[0][0] == word or bigram[0][1] == word:
            collocs.append(bigram[0][0] + " " + bigram[0][1])
    
    for colloc in collocs[:n]:
        print(colloc)

In [45]:
def get_top_n_trigram(scores, word, n):
    collocs = []
    for trigram in scores:
        if trigram[0][0] == word or trigram[0][1] == word or trigram[0][2] == word:
            collocs.append(trigram[0][0] + " " + trigram[0][1] + " " + trigram[0][2])
    
    for colloc in collocs[:n]:
        print(colloc)

In [46]:
samples = ['dishwasher', 'oven', 'refrigerator', 'cooktop', 'dryer']

In [47]:
for word in samples:
    print('\n\033[1mWord:' + ' ' + word + '\033[0m')
    get_top_n_bigram(bi_scores_ll , word, 5)


Word: dishwasher
bosch dishwasher
old dishwasher
new dishwasher
door dishwasher
dishwasher part

Word: oven
wall oven
convection oven
oven temperature
accurate oven
oven beeps

Word: refrigerator
refrigerator doors
refrigerator door
stainless refrigerator
refrigerator filter
bosch refrigerator

Word: cooktop
gas cooktop
induction cooktop
cooktop turns
glass cooktop
cooktop took

Word: dryer
dryer set
matching dryer
dryer sheets
vented dryer
conventional dryer


In [48]:
for word in samples:
    print('\n\033[1mWord:' + ' ' + word + '\033[0m')
    get_top_n_bigram(bi_scores_pmi , word, 5)


Word: dishwasher
710 dishwasher
designed dishwasher
dishwasher died
dishwasher everyday
dishwasher racks

Word: oven
accurate oven
oven beeps
oven thermometer
bisch oven
bright oven

Word: refrigerator
refrigerator doors
stainless refrigerator
refrigerator filter
refrigerator door
bosch refrigerator

Word: cooktop
burner cooktop
cooktop stays
30-inch cooktop
900.00 cooktop
advertised cooktop

Word: dryer
dryer sheets
220v dryer
corresponding dryer
accompanying dryer
companion dryer


In [49]:
for word in samples:
    print('\n\033[1mWord:' + ' ' + word + '\033[0m')
    get_top_n_trigram(tri_scores_student_t , word, 5)


Word: dishwasher
new bosch dishwasher
year old dishwasher
old bosch dishwasher
bosch ascenta dishwasher
dishwasher cleans dishes

Word: oven
self cleaning oven
true convection oven
convection wall oven
oven works great
oven consistently overcooks

Word: refrigerator
scratched refrigerator doors
ultraclarity refrigerator filter
stainless refrigerator started
bosch refrigerator filter
stainless steel refrigerator

Word: cooktop
bosch induction cooktop
bosch gas cooktop
stainless steel cooktop
white enamel cooktop
burner gas cooktop

Word: dryer
dryer drawing 10
dryer completely dried
washer dryer set
dryer keeps trying
absolute worst dryer


In [50]:
for word in samples:
    print('\n\033[1mWord:' + ' ' + word + '\033[0m')
    get_top_n_trigram(tri_scores_ll , word, 5)


Word: dishwasher
stainless steel dishwasher
800 series dishwasher
year old dishwasher
new bosch dishwasher
500 series dishwasher

Word: oven
oven works great
convection wall oven
inch wall oven
wall oven lasted
oven takes forever

Word: refrigerator
stainless steel refrigerator
scratched refrigerator doors
ultraclarity refrigerator filter
stainless refrigerator started
bosch refrigerator filter

Word: cooktop
stainless steel cooktop
800 series cooktop
bosch induction cooktop
bosch gas cooktop
burner gas cooktop

Word: dryer
nexxt 500plus dryer
dryer drawing 10
dryer completely dried
dryer takes forever
washer dryer set


Стоит отметить, что с названиями бытовой техники немного тяжело выделять именнованные сущности, как минимум из-за того, что многие товары - разного рода запчасти для собственно посудомоек, стиралок, сушилок, холодильников и тп и их названия часто состоят из длинных цепочек простых существительных. Например - "Bosch Dishwasher Center Spray Arm And Tube", "Washer Cold Water Inlet Valve", "Stainless Steel Counter Depth Bottom Freezer Refrigerator". Надо либо набирать очень длинные н-граммы, либо каким-то образом выделить правильные последовательности, каким-то образом не путать сам товар и куча его дескрипторов.